In [3]:
# 패키지 준비
import requests
from bs4 import BeautifulSoup

In [5]:
# html 요청
resp = requests.get("http://dhlottery.co.kr/gameResult.do?method=byWin")

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="EUC-KR'

In [6]:
# 응답 확인
resp.text[:100]

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="EUC-KR'

In [7]:
# 응답 HTML을 parsing 해서 객체 트리 구성
soup = BeautifulSoup(resp.text, "html.parser")

In [8]:
# 번호 정보가 있는 위치를 css 선택자로 지정해서 객체 목록 반환
num_spans = soup.select("div.nums span")
print(num_spans)

[<span class="ball_645 lrg ball1">3</span>, <span class="ball_645 lrg ball1">8</span>, <span class="ball_645 lrg ball2">17</span>, <span class="ball_645 lrg ball2">20</span>, <span class="ball_645 lrg ball3">27</span>, <span class="ball_645 lrg ball4">35</span>, <span class="ball_645 lrg ball3">26</span>]


In [20]:
# 각 객체를 순회하면서 번호 데이터를 읽기
numbers = []
for num_span in num_spans:
    # print(num_span)
    # print(num_span.string)
    # print(num_span.get_text())
    numbers.append(int(num_span.get_text()))

numbers

[3, 8, 17, 20, 27, 35, 26]

In [18]:
# 회차 정보가 포함된 요소의 의치를 css 선택자로 지정해서 객체로 반환
rnd = soup.select_one('div.win_result strong')
rnd = rnd.get_text()

In [21]:
"[ {0} 당첨 결과 ] : {1}".format(rnd, numbers)

'[ 947회 당첨 결과 ] : [3, 8, 17, 20, 27, 35, 26]'

In [24]:
# 연습 : 메뉴 목록 가져오기
links = soup.select('div#gnb > ul > li > a')
# links
for link in links:
    print( link.get_text() )

복권구매
복권정보
당첨결과
판매점
이벤트
행복공감


In [38]:
# 회차 번호를 지정해서 post 방식으로 요청하고 응답 수신
resp = requests.post("https://dhlottery.co.kr/gameResult.do?method=byWin", data={ 'drwNo' : 940, 'dwrNoList' : 940 })

In [39]:
resp.text[:100]

'\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n\r\n<!DOCTYPE html>\r\n<html lang="ko">\r\n<head>\r\n<meta charset="EUC-KR'

In [40]:
# 응답 HTML을 parsing 해서 객체 트리 구성
soup = BeautifulSoup(resp.text, "html.parser")

# 번호 정보가 있는 위치를 css 선택자로 지정해서 객체 목록 반환
num_spans = soup.select("div.nums span")
#print(num_spans)

# 각 객체를 순회하면서 번호 데이터를 읽기
numbers = []
for num_span in num_spans:
    # print(num_span)
    # print(num_span.string)
    # print(num_span.get_text())
    numbers.append(int(num_span.get_text()))

numbers

[3, 15, 20, 22, 24, 41, 11]

In [41]:
# 여러 회차의 번호 Crawling

import time 

# numbers_dict = {}
numbers_list = []

for rnd in range(1, 11):
    # 회차 번호를 지정해서 post 방식으로 요청하고 응답 수신
    resp = requests.post("https://dhlottery.co.kr/gameResult.do?method=byWin", data={ 'drwNo' : rnd, 'dwrNoList' : rnd })

    # 응답 HTML을 parsing 해서 객체 트리 구성
    soup = BeautifulSoup(resp.text, "html.parser")

    # 번호 정보가 있는 위치를 css 선택자로 지정해서 객체 목록 반환
    num_spans = soup.select("div.nums span")

    # 각 객체를 순회하면서 번호 데이터를 읽기
    numbers = []
    for num_span in num_spans:
        numbers.append(int(num_span.get_text()))

    numbers_list.append(numbers)

    time.sleep(1)

In [42]:
numbers_list

[[10, 23, 29, 33, 37, 40, 16],
 [9, 13, 21, 25, 32, 42, 2],
 [11, 16, 19, 21, 27, 31, 30],
 [14, 27, 30, 31, 40, 42, 2],
 [16, 24, 29, 40, 41, 42, 3],
 [14, 15, 26, 27, 40, 42, 34],
 [2, 9, 16, 25, 26, 40, 42],
 [8, 19, 25, 34, 37, 39, 9],
 [2, 4, 16, 17, 36, 39, 14],
 [9, 25, 30, 33, 41, 44, 6]]

In [ ]:
# 아래는 수집한 데이터를 데이터베이스에 저장하는 구현입니다.

In [1]:
# 패키지 준비
import requests
from bs4 import BeautifulSoup

In [2]:
# 여러 회차의 번호 Crawling

import time 

# numbers_dict = {}
numbers_list = []

for rnd in range(1, 11):
    # 회차 번호를 지정해서 post 방식으로 요청하고 응답 수신
    resp = requests.post("https://dhlottery.co.kr/gameResult.do?method=byWin", data={ 'drwNo' : rnd, 'dwrNoList' : rnd })

    # 응답 HTML을 parsing 해서 객체 트리 구성
    soup = BeautifulSoup(resp.text, "html.parser")

    # 번호 정보가 있는 위치를 css 선택자로 지정해서 객체 목록 반환
    num_spans = soup.select("div.nums span")

    # 각 객체를 순회하면서 번호 데이터를 읽기
    numbers = [rnd] # 첫 번째 데이터는 회차 번호
    for num_span in num_spans:
        numbers.append(int(num_span.get_text())) # 1등 당첨번호 + 보너스 번호

    numbers_list.append(numbers)

    time.sleep(1)

In [3]:
numbers_list

[[1, 10, 23, 29, 33, 37, 40, 16],
 [2, 9, 13, 21, 25, 32, 42, 2],
 [3, 11, 16, 19, 21, 27, 31, 30],
 [4, 14, 27, 30, 31, 40, 42, 2],
 [5, 16, 24, 29, 40, 41, 42, 3],
 [6, 14, 15, 26, 27, 40, 42, 34],
 [7, 2, 9, 16, 25, 26, 40, 42],
 [8, 8, 19, 25, 34, 37, 39, 9],
 [9, 2, 4, 16, 17, 36, 39, 14],
 [10, 9, 25, 30, 33, 41, 44, 6]]

In [4]:
import pymysql # 데이터베이스 연동 모듈 준비

In [8]:
# 1. 연결하기
conn = pymysql.connect(host='localhost',
                       database='pydemodb',
                       user='root',
                       password='Pa$$w0rd',
                       charset='utf8')

# 2. 명령하기
with conn.cursor() as cursor:
    sql = """
        create table tbl_lotto(
            rnd int primary key,
            no1 int not null,
            no2 int not null,
            no3 int not null,
            no4 int not null,
            no5 int not null,
            no6 int not null,
            bno int not null
        )
    """
    cursor.execute(sql)

conn.close()



In [9]:
# 1. 연결하기
conn = pymysql.connect(host='localhost',
                       database='pydemodb',
                       user='root',
                       password='Pa$$w0rd',
                       charset='utf8')

# 2. 명령하기
with conn.cursor() as cursor:
    sql = """
        insert into tbl_lotto (rnd, no1, no2, no3, no4, no5, no6, bno) 
        values (%s, %s, %s, %s, %s, %s, %s, %s)
    """
    # for numbers in numbers_list:
    #     cursor.execute(sql, numbers)

    cursor.executemany(sql, numbers_list)

    conn.commit()

conn.close()

In [15]:
# 아래 코드는 수집된 데이터를 파일로 저장하는 구현

import pandas as pd

numbers_df = pd.DataFrame(numbers_list, columns=['rnd', 'no1', 'no2', 'no3', 'no4', 'no5', 'no6', 'bno'])
# numbers_df

numbers_df.to_csv('data-files/lotto-numbers.csv', index=False)